**Python Code for Symptom Classification**

In [ ]:
import spacy

# Load a medical NLP model
nlp = spacy.load("en_core_web_sm")

# Example symptom-to-specialist mapping
symptom_to_specialist = {
    "chest pain": "Cardiologist",
    "fever": "General Physician",
    "headache": "Neurologist",
}

def predict_specialist(symptom):
    doc = nlp(symptom.lower())
    for word in doc:
        if word.text in symptom_to_specialist:
            return symptom_to_specialist[word.text]
    return "General Physician"

# Test input
symptom = "I have chest pain and dizziness"
print(predict_specialist(symptom))  # Output: Cardiologist


In [2]:
import pandas as pd

# Example doctor schedule
doctor_schedule = pd.DataFrame([
    {"doctor_id": "D001", "specialization": "Cardiologist", "available_days": ["Mon", "Wed", "Fri"], "available_slots": ["10:00 AM", "11:00 AM", "2:00 PM"]},
    {"doctor_id": "D002", "specialization": "Neurologist", "available_days": ["Tue", "Thu"], "available_slots": ["1:00 PM", "3:00 PM", "5:00 PM"]},
])

def find_available_doctor(specialization, preferred_day):
    available_doctors = doctor_schedule[doctor_schedule["specialization"] == specialization]

    for _, row in available_doctors.iterrows():
        if preferred_day in row["available_days"]:
            return row["doctor_id"], row["available_slots"][0]  # Return first available slot

    return None, None  # No available doctor found

# Example test
specialization_needed = "Cardiologist"
preferred_day = "Mon"
doctor_id, slot = find_available_doctor(specialization_needed, preferred_day)

print(f"Recommended Doctor: {doctor_id}, Available Slot: {slot}")


Recommended Doctor: D001, Available Slot: 10:00 AM


**AI-Powered Symptom Checker**

In [3]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import pickle

# Load NLP Model
nlp = spacy.load("en_core_web_sm")

# Example dataset
data = {
    "Symptoms": ["fever cough", "chest pain", "headache"],
    "Condition": ["Flu", "Heart Disease", "Migraine"],
    "Specialization": ["General Physician", "Cardiologist", "Neurologist"]
}

df = pd.DataFrame(data)

# Convert symptoms into numerical data
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df["Symptoms"])
y = df["Condition"]

# Train a simple Naive Bayes classifier
model = MultinomialNB()
model.fit(X, y)

# Save model & vectorizer
pickle.dump(model, open("symptom_checker.pkl", "wb"))
pickle.dump(vectorizer, open("vectorizer.pkl", "wb"))


/usr/local/lib/python3.11/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [ ]:
from flask import Flask, request, jsonify
import pickle

app = Flask(__name__)

# Load trained model
model = pickle.load(open("symptom_checker.pkl", "rb"))
vectorizer = pickle.load(open("vectorizer.pkl", "rb"))

# Condition-Specialist Mapping
condition_specialist = {
    "Flu": "General Physician",
    "Heart Disease": "Cardiologist",
    "Migraine": "Neurologist"
}

@app.route("/diagnose", methods=["POST"])
def diagnose():
    data = request.get_json()
    symptoms = data["symptoms"]

    # Convert symptoms to numerical data
    symptoms_vector = vectorizer.transform([symptoms])

    # Predict the condition
    predicted_condition = model.predict(symptoms_vector)[0]

    # Get doctor specialization
    specialist = condition_specialist.get(predicted_condition, "General Physician")

    return jsonify({"condition": predicted_condition, "recommended_doctor": specialist})

if __name__ == "__main__":
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
import requests

# API endpoint
url = "http://127.0.0.1:5000/diagnose"

# Test symptoms
symptoms = "I have chest pain and shortness of breath"

# Make a POST request
response = requests.post(url, json={"symptoms": symptoms})

# Print response
print(response.json())
